In [1]:
import cv2
import numpy as np

In [ ]:
class scan:
    def __init__(self, oriImg) :
        self.oriImg = oriImg


    def adjust(img):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (3, 3), 0) # 이미지를 흐리게 처리함 (noise 제거를 위해 사용)
        edged = cv2.Canny(gray, 75, 250) # edged를 검출하는 함수 (img, minVal, maxVal)
        contours, hierarchy = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        draw = img.copy()
        cv2.drawContours(draw,  contours, -1, (0, 255, 0))
        # 크기순으로 컨투어 정렬
        contours = sorted(contours, key=cv2.contourArea, reverse=True)[:5]
        for c in contours:
            peri = cv2.arcLength(c, True) # 외곽선 길이
            # print(peri)
            verticles = cv2.approxPolyDP(c, 0.02 * peri, True) # 외곽선 근사화
            if len(verticles) == 4 : 
                break
        pts = verticles.reshape(4, 2) # 배열을 4 * 2 크기로 조정

